# MediaTek Genio AI 開發指南-1：TensorFlow Lite 模型效能測試

## 🎯 關於這個筆記本
這是一個完整的教學範例，教您如何在 MediaTek Genio SoC（系統晶片）上使用 ArmNN 運行 AI 模型，並測量其效能。

## 📚 背景知識：
- **TensorFlow Lite**：Google 開發的輕量級 AI 推論框架，專為手機、嵌入式裝置設計，支援`.tflite`格式的AI模型。
- **ArmNN**：Arm 公司開發的神經網路推論引擎，可以加速推論框架背景的 AI計算工作
- **MediaTek Genio SoC**：聯發科的 AI 處理器，內建與ArmNN相容的 Cortex-A CPU、Mali-GPU

## 📋 您將學會：
- 如何透過TFLite Runtime載入和配置 AI 模型
- 如何使用 ArmNN於GPU 或 CPU 上進行 AI 推論加速
- 如何測量 AI 模型的執行速度和記憶體使用量
- 如何將這些技術整合到您的應用程式中

## 1. 環境設置

我們需要切換到專案的根目錄，這樣程式才能找到模型檔案和其他必要的資源。

In [ ]:
% cd ..

## 2. 匯入必要程式庫

載入Tensorflow及 Python 程式庫來處理 AI 推論的工作流程。

In [ ]:
import time, warnings, psutil, os
import numpy as np
import tensorflow as tf
from tqdm import tqdm
warnings.simplefilter('ignore')

print(f"TensorFlow 版本: {tf.__version__}")
print(f"NumPy 版本: {np.__version__}")

## 3. 設定測試參數

根據您的環境及測試需求，設定要使用的加速器、AI模型以及ArmNN。
- **device**：加速器類型，可選擇 'CpuAcc' 或 'GpuAcc'
- **library_path**：ArmNN delegate 程式庫路徑
- **tflite_path**：TensorFlow Lite 模型檔案路徑

In [ ]:
device = 'GpuAcc'
tflite_path = "models/yolov8n_float32.tflite"
library_path = "/ArmNN-linux-aarch64/libarmnnDelegate.so"

## 4. 載入 ArmNN Delegate

載入 ArmNN 加速器來提升 AI 模型推論速度。
1. 載入 ArmNN delegate 程式庫
2. 設定使用的運算後端（CPU 或 GPU）
3. 配置日誌級別以控制輸出訊息

In [ ]:
armnn_delegate = tf.lite.experimental.load_delegate(
    library=library_path,
    options={"backends": device, "logging-severity": "fatal"}
)
print("✅ ArmNN delegate 載入成功！")
print(f"🚀 使用後端: {device}")

## 5. 初始化 TensorFlow Lite 解釋器

建立 TensorFlow Lite 解釋器並配置 AI 模型。
1. 載入 .tflite 模型檔案並綁定 ArmNN 加速器
2. 分配模型所需的記憶體空間
3. 取得模型的輸入和輸出資訊

In [ ]:
interpreter = tf.lite.Interpreter(
    model_path=tflite_path, 
    experimental_delegates=[armnn_delegate]
)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(f"✅ 模型載入成功！")
print(f"輸入張量形狀: {input_details[0]['shape']}")
print(f"輸入資料型態: {input_details[0]['dtype']}")
print(f"輸出張量形狀: {output_details[0]['shape']}")

## 6. 準備輸入資料

為 AI 模型準備測試用的輸入資料。
1. 根據模型要求的形狀產生隨機數字陣列
2. 轉換資料型態以符合模型規格
3. 將準備好的資料送入模型

In [ ]:
inputs = np.random.rand(*input_details[0]['shape']).astype(input_details[0]['dtype'])

print("✅ 輸入資料準備完成！")
print(f"輸入資料形狀: {inputs.shape}")
print(f"輸入資料型態: {inputs.dtype}")

interpreter.set_tensor(input_details[0]['index'], inputs)

## 7. 執行推論測試

重複執行多次 AI 推論並測量效能。
1. 使用 tqdm 進度條執行指定次數的推論
2. 記錄推論的開始和結束時間
3. 取得模型的輸出結果

In [ ]:
iteration = 10
print(f"開始執行 {iteration} 次推論測試...")
inference_start_time = time.time()

for i in tqdm(range(iteration), desc="推論進度", unit="次"):
    interpreter.invoke()

inference_end_time = time.time()
outputs = interpreter.get_tensor(output_details[0]['index'])

## 8. 顯示測試結果

計算並顯示 AI 模型的效能指標。
1. 計算平均單次推論時間（毫秒）
2. 計算推論吞吐量（每秒處理次數）
3. 以表格格式顯示完整的效能報告

In [ ]:
total_inference_time = inference_end_time - inference_start_time
avg_inference_time_ms = total_inference_time * 1000 / iteration

print('=' * 50)
print('TensorFlow Lite 模型效能測試結果')
print('=' * 50)
print(f'單次推論時間: {avg_inference_time_ms:.2f} ms')
print(f'推論吞吐量: {1000 / avg_inference_time_ms:.2f} FPS')
print('=' * 50)

## 9. 整合到實際應用

您可以參考以下的程式範例，將ArmNN加速方案整合到您的AI工作流程中。

```python
import tensorflow as tf

interpreter = tf.lite.Interpreter(
    model_path="<path to your tflite model>",
    experimental_delegates=[
        armnn_delegate = tf.lite.experimental.load_delegate(
            library="<path to libarmnnDelegate.so>",
            options={"backends": "<CpuAcc or GpuAcc>", "logging-severity": "fatal"}
        )
    ]
)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])
```